In [12]:
from openai import OpenAI
import pandas as pd
from dotenv import load_dotenv
import os
client = OpenAI()

# Set your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")
reviews_df = pd.read_csv('datasets/full-reviews.csv')

# Send the prompt to ChatGPT
def chat_gpt(review):
    response = client.chat.completions.create(
    model="gpt-4o",
    response_format={ "type": "text" },
    messages=[
    {"role": "system", "content": "You are a helpful assistant designed to summarize reviews and return a short **summary**, a short **sentiment analysis**, and a **numerical rating** (a single number, not a fraction) of the sentiment from 0-100 with 100 being the most positive."},
    {"role": "user", "content": f"Summarize this review: '{review}"},
  ]
)
    return response.choices[0].message.content


In [13]:
specific_movie_reviews = reviews_df[reviews_df['movie_title'] == "Percy Jackson & the Olympians: The Lightning Thief"]

In [14]:
specific_movie_reviews.head()

,movie_title,review_content,review_type
0,Percy Jackson & the Olympians: The Lightning T...,A fantasy adventure that fuses Greek mythology...,Fresh
1,Percy Jackson & the Olympians: The Lightning T...,"Uma Thurman as Medusa, the gorgon with a coiff...",Fresh
2,Percy Jackson & the Olympians: The Lightning T...,With a top-notch cast and dazzling special eff...,Fresh
3,Percy Jackson & the Olympians: The Lightning T...,Whether audiences will get behind The Lightnin...,Fresh
4,Percy Jackson & the Olympians: The Lightning T...,What's really lacking in The Lightning Thief i...,Rotten


In [16]:
specific_movie_reviews.shape

(149, 3)

In [23]:
# Initialize a list to hold the results
results = []

for index, review in enumerate(specific_movie_reviews['review_content']):
    response = chat_gpt(review)
    # Assuming the response is in the format "Summary: ... Sentiment: ... Rating: ..."
    lines = response.split('\n')
    
    summary = lines[0].split(':', 1)[1].strip() if len(lines) > 0 else "N/A"
    sentiment = lines[1].split(':', 1)[1].strip() if len(lines) > 1 else "N/A"
    rating = lines[2].split(':', 1)[1].strip() if len(lines) > 2 else "N/A"
    
    results.append({
        'Review Index': index,
        'Summary': summary,
        'Sentiment': sentiment,
        'Rating': rating
    })

# Convert the list to a DataFrame
df_results = pd.DataFrame(results)

IndexError: list index out of range

In [19]:
reviews_df['movie_title'].value_counts(ascending=False)

movie_title
Star Wars: The Rise of Skywalker        992
Solo: A Star Wars Story                 948
Star Wars: The Last Jedi                946
Rogue One: A Star Wars Story            892
Spider-Man: Far From Home               880
                                       ... 
The Influence (La influencia)             5
Thick as Thieves (The Code)               5
Adventures of a Teenage Dragonslayer      4
You Get Me                                4
Farewell                                  1
Name: count, Length: 17100, dtype: int64